In [26]:
import numpy as np
import argparse
import cv2
import os


Credits: 
	1. https://github.com/opencv/opencv/blob/master/samples/dnn/colorization.py
	2. http://richzhang.github.io/colorization/
	3. https://github.com/richzhang/colorization/


In [27]:
DIR = r"C:\Users\P Srivalli Patel\Downloads\dataset"
PROTOTXT = os.path.join(DIR, r"Model/colorization_deploy_v2.prototxt")
POINTS = os.path.join(DIR, r"Model/pts_in_hull.npy")
MODEL = os.path.join(DIR, r"Model/colorization_release_v2.caffemodel")



In [28]:
TestImagePath="C:/Users/P Srivalli Patel/Downloads/dataset/test_dataset/"

In [29]:
# Load the Model
print("Load model")
net = cv2.dnn.readNetFromCaffe(PROTOTXT, MODEL)
pts = np.load(POINTS)

Load model


In [30]:
# Load centers for ab channel quantization used for rebalancing.
class8 = net.getLayerId("class8_ab")
conv8 = net.getLayerId("conv8_313_rh")
pts = pts.transpose().reshape(2, 313, 1, 1)
net.getLayer(class8).blobs = [pts.astype("float32")]
net.getLayer(conv8).blobs = [np.full([1, 313], 2.606, dtype="float32")]


In [34]:
# Load the input image
ImagePath=TestImagePath+"2.jpg"
print(ImagePath)
image = cv2.imread(ImagePath)
scaled = image.astype("float32") / 255.0
lab = cv2.cvtColor(scaled, cv2.COLOR_BGR2LAB)

resized = cv2.resize(lab, (224, 224))
L = cv2.split(resized)[0]
L -= 50

C:/Users/P Srivalli Patel/Downloads/dataset/test_dataset/2.jpg


In [35]:
print("Colorizing the image")
net.setInput(cv2.dnn.blobFromImage(L))
ab = net.forward()[0, :, :, :].transpose((1, 2, 0))

ab = cv2.resize(ab, (image.shape[1], image.shape[0]))

Colorizing the image


In [37]:
L = cv2.split(lab)[0]
colorized = np.concatenate((L[:, :, np.newaxis], ab), axis=2)

colorized = cv2.cvtColor(colorized, cv2.COLOR_LAB2BGR)
colorized = np.clip(colorized, 0, 1)

colorized = (255 * colorized).astype("uint8")

cv2.imshow("Original", image)
cv2.imshow("Colorized", colorized)
cv2.waitKey(0)

-1